In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium . webdriver import Chrome
from selenium . webdriver . chrome . options import Options
from selenium . webdriver . chrome . service import Service
import os
import json
import lxml 
import time

In [2]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
l = 5
URL = "https://codeforces.com/problemset/page/{i}"
CODEFORCES_URL = "https://codeforces.com"   
from config import PROBLEM_DIR, METADATA_DIR, EDITORIAL_DIR


In [3]:
import undetected_chromedriver as uc
driver = uc.Chrome(version_main=131,use_subprocess=False)
driver.get(URL)
print(driver.title)
time.sleep(5)
driver.quit()

Problemset - Codeforces


In [4]:
from urllib.request import Request, urlopen

# req = Request(
#     url=URL, 
#     headers={'User-Agent': 'Mozilla/5.0'}
# )
# webpage = urlopen(req).read()

In [5]:
# soup = BeautifulSoup(webpage,"lxml")
# problem_table = soup.find("table",class_="problems")
Questions_list = []
for i in range(1, l + 1):
    formatted_url = URL.format(i=i)  # Format the URL with the page number
    req = Request(
        url=formatted_url, 
        headers={'User-Agent': 'Mozilla/5.0'}
    )
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, "lxml")
    problem_table = soup.find("table", class_="problems")
    
    if problem_table:
        table_row = problem_table.find_all("tr")
        for tr in table_row[1:]:
            td = tr.find_all("td")
            if len(td) >= 4:  # Make sure there are enough columns in the row
                Problems = {
                    "#": ' '.join(td[0].text.split()),
                    "Name": ' '.join(td[1].text.split()),
                    "Rating": ' '.join(td[3].text.split()),
                }
                Questions_list.append(Problems)
    else:
        print(f"Problem table not found on page {i}!")

print(Questions_list)
print(len(Questions_list))



[{'#': '2063F2', 'Name': 'Counting Is Not Fun (Hard Version) combinatorics, data structures, dfs and similar, dsu, graphs, implementation, trees', 'Rating': ''}, {'#': '2063F1', 'Name': 'Counting Is Not Fun (Easy Version) combinatorics, data structures, dfs and similar, dp, dsu, graphs, implementation, math, trees', 'Rating': ''}, {'#': '2063E', 'Name': 'Triangle Tree constructive algorithms, data structures, dfs and similar, dp, dsu, graphs, greedy, implementation, math, trees', 'Rating': ''}, {'#': '2063D', 'Name': 'Game With Triangles binary search, data structures, geometry, greedy, math, ternary search, two pointers', 'Rating': ''}, {'#': '2063C', 'Name': 'Remove Exactly Two brute force, data structures, dfs and similar, dp, graphs, greedy, sortings, trees', 'Rating': ''}, {'#': '2063B', 'Name': 'Subsequence Update constructive algorithms, data structures, greedy, sortings', 'Rating': ''}, {'#': '2063A', 'Name': 'Minimal Coprime math, number theory', 'Rating': ''}, {'#': '2061I', 

In [6]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from IPython.display import display, Math

In [7]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

BASE_URL = "https://codeforces.com/problemset/problem"

def Fetch_problem(id, name):
    Problem_object = {}
    id = id.strip()
    contest_id = id[:4]
    alpha = id[4:]
    Url = f"{BASE_URL}/{contest_id}/{alpha}"
    pro_req = Request(
        url=Url,
        headers={'User-Agent': 'Mozilla/5.0'}
    )
    
    try:
        # Fetch the problem page
        problem_page = urlopen(pro_req).read()
        soup_pro = BeautifulSoup(problem_page, 'html.parser')
        problem_page_content = soup_pro.find("div", id="body")

        if not problem_page_content:
            print("Problem page content not found.")
            return {}

        # Extract metadata
        metadata = problem_page_content.find("div", class_="header")
        if metadata:
            title = metadata.find("div", class_="title").text.strip()
            time_limit = metadata.find("div", class_="time-limit").text.strip()
            memory_limit = metadata.find("div", class_="memory-limit").text.strip()
        else:
            print("Metadata not found.")
            title = time_limit = memory_limit = None

        # Extract problem statement
        problem_statement_div = problem_page_content.find("div", class_="problemindexholder")
        Problem_question = problem_statement_div.get_text("\n", strip=True) if problem_statement_div else None
        
        # Extract sidebar content
        sidebar_content = problem_page_content.find("div", id="sidebar")
        Tutorial_link = None
        Announcement_link = None
        
        if sidebar_content:
            list_blog_url = sidebar_content.find_all("li")
            for li in list_blog_url:
                link_tag = li.find("a")
                if link_tag:
                    link_text = li.text.strip()
                    if link_text == "Tutorial (en)":
                        Tutorial_link = link_tag["href"]
                    elif link_text == "Announcement":
                        Announcement_link = link_tag["href"]
        else:
            print("Sidebar content not found.")
        
        # Build the problem object
        Problem_object = {
            "Problem_statement": Problem_question,
            "Tutorial_link": Tutorial_link,
            "Announcement_link": Announcement_link,
            "Metadata": {
                "Title": title,
                "Time Limit": time_limit,
                "Memory Limit": memory_limit
            }
        }

    except Exception as e:
        print(f"Error fetching problem data: {e}")
        return {}

    return Problem_object

# Test the function
problem_data = Fetch_problem("2053I2", "Affectionate Arrays (Hard Version)")

print(problem_data)


def Fetch_editorial(Tutorial_Link, problem_id):
    if not Tutorial_Link:  # Check if Tutorial_Link is None or empty
        with open("./data/logs/process_log.txt", "a", encoding="utf-8") as log_file:
            log_file.write(f"No tutorial link found for ID: {problem_id}. Skipping editorial fetch.\n")
        return None  # Skip fetching the editorial if no link is provided
    LINK_URL = CODEFORCES_URL + Tutorial_Link
    editorial_page = urlopen(Request(url=LINK_URL, headers={"User-Agent": "Mozilla/5.0"})).read()
    soup = BeautifulSoup(editorial_page, "html.parser")

    # Locate the paragraph with the problem ID
    p = soup.find("p", string=lambda text: text and problem_id in text)
    if not p:
        print("Problem ID paragraph not found.")
        return None

    spoiler_divs = []
    sibling = p.find_next_sibling()
    found = False
    while sibling:
        # Check if it's a <p> tag directly, terminate traversal if so
        if sibling.name == "p" and  found:
            sibling = sibling.find_next_sibling()
            break

        # Check if it's a <div> with a class "spoiler"
        if sibling.name == "div" and "spoiler" in sibling.get("class", []):
            # Ensure no <p> tag inside this <div>
            # if not sibling.find("p"):
            found = True
            spoiler_divs.append(sibling.get_text("\n", strip=True))

        # Move to the next sibling
        sibling = sibling.find_next_sibling()

    return spoiler_divs

editorial_data = Fetch_editorial("/blog/entry/136455", "2053H")
print(editorial_data)

{'Problem_statement': "The problem statement has recently been changed.\nView the changes.\n×\nI2. Affectionate Arrays (Hard Version)\ntime limit per test\n3 seconds\nmemory limit per test\n512 megabytes\ninput\nstandard input\noutput\nstandard output\nNote that this statement is different to the version used in the official round. The statement has been corrected to a solvable version. In the official round, all submissions to this problem have been removed.\nThis is the hard version of the problem. The difference between the versions is that in this version, you need to compute the sum of value of different arrays. You can hack only if you solved all versions of this problem.\nIris treasures an integer array $$$a_1, a_2, \\ldots, a_n$$$. She knows this array has an interesting property: the maximum absolute value of all elements is less than or equal to the sum of all elements, that is, $$$\\max(\\lvert a_i\\rvert) \\leq \\sum a_i$$$.\nIris defines the\nboredom\nof an array as its ma

In [8]:
def save_problem_and_metadata(problem_data, editorial_data, problem_id, log_file_path):
    # Define directories
    problems_dir = "./data/problems"
    metadata_dir = "./data/metadata"
    editorial_dir = "./data/editorials"
    log_dir = "./data/logs"

    # Create directories if they don't exist
    os.makedirs(problems_dir, exist_ok=True)
    os.makedirs(metadata_dir, exist_ok=True)
    os.makedirs(editorial_dir, exist_ok=True)
    os.makedirs(log_dir,exist_ok=True)

    try:
        # Save problem statement
        problem_file_path = os.path.join(problems_dir, f"{problem_id}.txt")
        with open(problem_file_path, "w", encoding="utf-8") as problem_file:
            problem_file.write(problem_data.get("Problem_statement", "Problem statement not available."))

        # Save metadata
        metadata_file_path = os.path.join(metadata_dir, f"{problem_id}.json")
        with open(metadata_file_path, "w", encoding="utf-8") as metadata_file:
            json.dump(problem_data.get("Metadata", {}), metadata_file, indent=4)

        # Save editorial content
        editorial_file_path = os.path.join(editorial_dir, f"{problem_id}_editorial.txt")
        with open(editorial_file_path, "w", encoding="utf-8") as editorial_file:
            for idx, content in enumerate(editorial_data, start=1):
                editorial_file.write(f"Section {idx}:\n{content}\n\n")

        # Write success log
        with open(log_file_path, "a", encoding="utf-8") as log_file:
            log_file.write(f"Problem ,Editorial and metadata saved successfully for ID: {problem_id}\n")
    except Exception as e:
        # Write error log
        with open(log_file_path, "a", encoding="utf-8") as log_file:
            log_file.write(f"Error saving data for ID: {problem_id}. Error: {str(e)}\n")


# Example usage
def main():
    log_dir = "./data/logs"
    os.makedirs(log_dir, exist_ok=True)
    log_file_path = os.path.join(log_dir, "process_log.txt")

    for problem in Questions_list:
        problem_id = problem['#']
        problem_name = problem['Name']
        problem_data = Fetch_problem(problem_id, problem_name)

        if problem_data:
            editorial_data = Fetch_editorial(problem_data.get("Tutorial_link"), problem_id)
            if editorial_data:
                try:
                    save_problem_and_metadata(problem_data, editorial_data, problem_id, log_file_path)
                except Exception as e:
                    # Log and continue on error in saving data
                    with open(log_file_path, "a", encoding="utf-8") as log_file:
                        log_file.write(f"Unhandled error saving data for ID: {problem_id}. Skipping to the next question.\nError: {str(e)}\n")
            else:
                # Log missing editorial data
                with open(log_file_path, "a", encoding="utf-8") as log_file:
                    log_file.write(f"Editorial data not found for ID: {problem_id}. Skipping to the next question.\n")
        else:
            # Log missing problem data
            with open(log_file_path, "a", encoding="utf-8") as log_file:
                log_file.write(f"Problem data not found for ID: {problem_id}. Skipping to the next question.\n")


# Call the main function
main()


Problem ID paragraph not found.
Problem ID paragraph not found.
Problem ID paragraph not found.
Problem ID paragraph not found.
Problem ID paragraph not found.
Problem ID paragraph not found.
Problem ID paragraph not found.
Problem ID paragraph not found.
Problem ID paragraph not found.


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Problem page content not found.


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Problem page content not found.


...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Problem page content not found.


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Problem page content not found.


HTTPError: HTTP Error 404: 